# Data acquisition

I will use data sources such as:

• List of Toronto postal codes from [WIKIPEDIA.](https://https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

• Geospatial Coordinates CSV file for Toronto postal codes from http://cocl.us/Geospatial_data.

• Foursquare API - to explore venues in Toronto neighbourhoods

I'll use the Toronto postal code list from Wikipedia. In this list every neighborhood of Toronto is assigned a corresponding postcode. Then I will join a csv file with geolocation data to this list. After connecting, each neighborhood will be assigned the appropriate latitude and longitude value. Finally I will use Foursquare API to explore these districts. The collected data will allow me to solve the task.

## Below I present the process of obtaining raw data and processing it into data to be used for analysis.

#### Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from bs4 import BeautifulSoup

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##### Setting pandas options

In [0]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

----------------------

## 1. Downloading the Toronto postcode table from Wikipedia and converting it to Pandas Data Frame

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
print(page.status_code)

200


#### Creating BeautifulSoup instance

In [0]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

#### Searching for table in wikipedia site

In [0]:
tb = soup.find('table', class_='wikitable')

In [0]:
#for link in tb.find_all('tr'):
#   name = link.find('td')

#### Creating list with all table rows

In [0]:
t_row=tb.findAll('tr')
#t_row

In [8]:
t_row[5].findAll('td')  # view of single row

[<td>M5A
 </td>, <td>Downtown Toronto
 </td>, <td>Regent Park / Harbourfront
 </td>]

In [0]:
#@title Form 1
#list=[]
#for x in t_row[1:3]:
#    list.append(x)

In [0]:
#@title Form 2
#l2=[]
#for l in list:
#    l2.append(str(l).replace('<tr>',''))
#l2

#### Creating a list with clean data in rows

In [0]:
final=[]
for k in t_row[1:]:
    new=str(k).replace('<tr>\n<td>','').replace('\n</td>','').replace('</tr>','').split('\n<td>')
    final.append(new)

#### Checking the preview of the list items

In [12]:
final[0:3] 

[['M1A', 'Not assigned', ''],
 ['M2A', 'Not assigned', ''],
 ['M3A', 'North York', 'Parkwoods']]

#### Extracting columns names

In [13]:
columns_df=str(t_row[0]).replace('<tr>\n<th>','').replace('\n</th>','').replace('</tr>','').split('\n<th>')
columns_df

['Postal code', 'Borough', 'Neighborhood']

#### Creating raw data frame

In [14]:
table=pd.DataFrame(final, columns=columns_df)
table.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [15]:
table.shape

(180, 3)

#### Delete rows with blank entries in column "Borough"

In [16]:
table=table[table.Borough != 'Not assigned']
table.reset_index(inplace=True, drop=True)
table

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [17]:
table.shape

(103, 3)

#### Loop for replaceing "/" to ","

In [0]:
for i, t in enumerate(table['Neighborhood']): 
    t=t.replace(' /',',')
    table['Neighborhood'][i]=t
    #print(i,'---',table['Neighborhood'][i]) 

In [19]:
table.head() #Table preview after changes

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
table['Postal code'].value_counts().shape

(103,)

In [21]:
table[table['Neighborhood']==''].count()

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

In [22]:
print('My table has {} rows.'.format(table.shape[0]))

My table has 103 rows.


--------------------------------------------------------------------------------------------

# 2. Merging Toronto postal codes DataFrame and CSV file with geolocalisation data

In [0]:
cords=pd.read_csv('http://cocl.us/Geospatial_data')

In [24]:
cords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Creating final data frame

In [0]:
table_cords=pd.merge(table, cords, how='left', left_on='Postal code', right_on='Postal Code')
del table_cords['Postal Code']

In [26]:
table_cords.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Creating map of Toronto

In [27]:
address = 'Toronto, ONT'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.678523999999996, -79.62912913064454.


In [28]:
# creating map of Toronto using latitude and longitude values
map_toronto=folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(table_cords['Latitude'], table_cords['Longitude'], table_cords['Borough'], table_cords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Using the Foursquare API to explore neighborhoods

In [29]:
#path to file with Foursquare API credentials
cd '/content/drive/My Drive/Colab Notebooks/Coursera_Capstone'

/content/drive/My Drive/Colab Notebooks/Coursera_Capstone


In [0]:
credentials=list(pd.read_csv('credentials.csv'))

In [31]:
#@title Cell with hidden Foursquare API credentials

CLIENT_ID = credentials[0] # your Foursquare ID
CLIENT_SECRET = credentials[1] # your Foursquare Secret
VERSION = '20200430' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Credentials OK')

Credentials OK


In [0]:
LIMIT = 100
radius = 500

#### Function which getting top 100 venues that are in Toronto's neighborhoods within a radius of 500 meters from their centers.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue Id',
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
toronto_venues = getNearbyVenues(names=table_cords['Neighborhood'],
                                   latitudes=table_cords['Latitude'],
                                   longitudes=table_cords['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [35]:
print('toronto_venues DataFrame size: ',toronto_venues.shape)

toronto_venues DataFrame size:  (2110, 8)


In [36]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Id,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,4cb11e2075ebb60cd1c4caad,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,4c633acb86b6be9a61268e34,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,4bbe904a85fbb713420d7167,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,4f3ecce6e4b0587016b6f30d,Portugril,43.725819,-79.312785,Portuguese Restaurant
